In [1]:
model_name = "sam4096/qwen2tools:1.5b"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a LLMChain using with a Ollama model.
- Define a prompt template for summarization.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with a sample text and print the summary.


In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Load the Ollama model
llm = ChatOllama(model=model_name)

# Define the prompt template
prompt_template = ChatPromptTemplate([
    ("system", "Your task is to summarize a text in one sentence."),
    ("human", "{text}"),
])

# Create the LLMChain
chain = prompt_template | llm 

# Run the chain with a sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

summary = chain.invoke(text)
print(summary.content)

The text highlights the evolution of deep learning in processing and generating unstructured data like text, images, and video over the past decade and its impact on various industries, from large language models to transformational impacts across different sectors.


In [3]:
# Stream the chain output
for chunk in chain.stream({"text": text}):
    print(chunk.content, end="", flush=True)#

The text discusses how artificial intelligence, especially large language models (LLMs), has significantly advanced over the past decade, processing and generating unstructured data like text, images, and video. This advancement is expected to have significant impacts on businesses, societies, and individuals, driven by advancements in technology, high-profile applications, and potential for transformative effects across various sectors.

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

In [4]:
from langchain_core.tools import tool

# Create custom tool
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args) # -> definition of tool arguments

# Invoke custom tool
multiply.invoke({"first_int": 4, "second_int": 5})

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


20

In [5]:
from langchain_ollama.chat_models import ChatOllama

# Load the Ollama model
llm = ChatOllama(model=model_name)

# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke("whats 5 times forty two")
print(msg.tool_calls)

[{'name': 'multiply', 'args': {'first_int': 42, 'second_int': 5}, 'id': 'acb863f3-e7fc-4b51-8371-2aea32155474', 'type': 'tool_call'}]


In [9]:
# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply

# Run chain
chain.invoke("whats 5 times forty two")

210

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.

In [12]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Load the Ollama model
llm = ChatOllama(model=model_name)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{user_input}"), 
    ("placeholder", "{agent_scratchpad}"),
])


# Custom math tools
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add, exponentiate]

SyntaxError: invalid syntax (agent.py, line 1057)

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=False)

In [ ]:
agent_executor.invoke(
    {
        "user_input": "Take 3 to the fifth power then add that 12."
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 243}`


255Great job! Your calculation for 3 to the fifth power is correct. Now, adding that result with 12 gives us a total of 255.

> Finished chain.


{'user_input': 'Take 3 to the fifth power then add that 12.',
 'output': 'Great job! Your calculation for 3 to the fifth power is correct. Now, adding that result with 12 gives us a total of 255.'}

#### Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Load the Ollama model
llm = ChatOllama(model=model_name)

# Define memory object for conversation history
memory = ConversationBufferMemory(memory_key="chat_history")

# Add history placeholder to prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    MessagesPlaceholder("chat_history"),
    ("human", "{user_input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, return_intermediate_steps=True)

SyntaxError: invalid syntax (agent.py, line 1057)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

question = "what was the previous question?"
ai_msg_1 = agent_executor.invoke({"user_input": "what was the previous question?", "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "What was the question?"
ai_msg_2 = agent_executor.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

NameError: name 'agent_executor' is not defined